# Setup

In [1]:
# General libraries 
from datetime import date
from datetime import *
import pandas as pd
import numpy as np
import warnings
import logging
import mlflow
import sys
import os

# Plots
from matplotlib.offsetbox import TextArea, AnnotationBbox
import matplotlib.pyplot as plt
import networkx as nx

# Stats
from scipy.stats import norm

# LP
import pulp

# Directory

In [2]:
# Set cd to parent directory
os.chdir("/Users/ignasipascual/Documents/GitHub/Optimizer")
cwd = os.getcwd()
print("Current Working Directory:", cwd)

Current Working Directory: /Users/ignasipascual/Documents/GitHub/Optimizator


# Assets

In [3]:
# Load assets
from utils.simulator import Simulator
from utils.plotter import Plotter

# Parameters

In [4]:
# Set random seed
np.random.seed(42)

# Sales parameters
supply_params = (200, 50)
sell_in_params = (150, 40)

# Initial inventory
initial_inventory = 1000

# Start simulator

In [5]:
# Initialize simulator
sim = Simulator()

# Show
sim

## Simulate data

In [6]:
# Example usage:
plants = ['plant_1']
warehouses = ['warehouse_1', 'warehouse_2']
markets = ['market_1', 'market_2']
products = ['product_1']
weeks = [202501 + i for i in range(10)]

# Example warehouse-market mapping
warehouse_market_map = {
    "warehouse_1": ["market_1"], 
    "warehouse_2": ["market_1", "market_2"], 
}

In [7]:
# Generate the dataset
sc_data = sim.generate_supply_chain_data(plants, warehouses, markets, products, weeks, warehouse_market_map=warehouse_market_map)

# Show
sc_data.head()

,plant,warehouse,market,product,week
0,plant_1,warehouse_1,market_1,product_1,202501
3,plant_1,warehouse_1,market_1,product_1,202502
6,plant_1,warehouse_1,market_1,product_1,202503
9,plant_1,warehouse_1,market_1,product_1,202504
12,plant_1,warehouse_1,market_1,product_1,202505


In [8]:
# Printing distinct values
print("Distinct plants:", sc_data['plant'].unique())
print("Distinct warehouses:", sc_data['warehouse'].unique())
print("Distinct markets:", sc_data['market'].unique())
print("Distinct products:", sc_data['product'].unique())

Distinct plants: ['plant_1']
Distinct warehouses: ['warehouse_1' 'warehouse_2']
Distinct markets: ['market_1' 'market_2']
Distinct products: ['product_1']


## Generate flows

In [9]:
# Simulate data
sc_data = sim.simulate_data_normal(
    supply_params,
    sell_in_params
)

# Show
sc_data.head()

,plant,warehouse,market,product,week,supply,sell_in
0,plant_1,warehouse_1,market_1,product_1,202501,225.0,126.0
3,plant_1,warehouse_1,market_1,product_1,202502,193.0,224.0
6,plant_1,warehouse_1,market_1,product_1,202503,232.0,149.0
9,plant_1,warehouse_1,market_1,product_1,202504,276.0,108.0
12,plant_1,warehouse_1,market_1,product_1,202505,188.0,183.0


## Inventory

In [10]:
# Calculate inventory
sc_data = sim.calculate_rolling_inventory(initial_inventory)

# Show
sc_data.head()

,plant,warehouse,market,product,week,supply,sell_in,inventory
0,plant_1,warehouse_1,market_1,product_1,202501,225.0,126.0,1099.0
3,plant_1,warehouse_1,market_1,product_1,202502,193.0,224.0,1068.0
6,plant_1,warehouse_1,market_1,product_1,202503,232.0,149.0,1151.0
9,plant_1,warehouse_1,market_1,product_1,202504,276.0,108.0,1319.0
12,plant_1,warehouse_1,market_1,product_1,202505,188.0,183.0,1324.0


## Lead times

In [11]:
# Add lead times
sc_data_lead_times = sim.set_lead_times()

# Show
sc_data_lead_times.head()

,plant,warehouse,market,product,week,supply,sell_in,inventory,lead_time
0,plant_1,warehouse_1,market_1,product_1,202501,225.0,126.0,1099.0,4
1,plant_1,warehouse_1,market_1,product_1,202502,193.0,224.0,1068.0,4
2,plant_1,warehouse_1,market_1,product_1,202503,232.0,149.0,1151.0,4
3,plant_1,warehouse_1,market_1,product_1,202504,276.0,108.0,1319.0,4
4,plant_1,warehouse_1,market_1,product_1,202505,188.0,183.0,1324.0,4


## Summary

In [12]:
# Get summary
summary = sim.get_summary()

# Show
summary

inventory                         supply                       \
                  min     max    mean     std    min    max   mean    std   
warehouse                                                                   
warehouse_1    1068.0  1847.0  1422.7  282.66  177.0  279.0  222.3  36.10   
warehouse_2     997.0  1520.0  1266.4  182.01  104.0  273.0  174.7  40.93   

            sell_in                      lead_time                 
                min    max   mean    std       min max mean   std  
warehouse                                                          
warehouse_1    72.0  224.0  137.6  45.69         4   4  4.0  0.00  
warehouse_2    79.0  192.0  148.9  32.83         3  10  6.5  3.59

# Save data

In [13]:
# Write csv
sc_data_lead_times.to_csv('data/simulated_data.csv', index=False)